In [1]:
import pyodbc
import etl_dwh_loader as etl_loader  
import sys
import importlib

# Si vous modifiez etl_dwh_loader.py, ré-exécutez cette cellule
# pour forcer le rechargement du module
importlib.reload(etl_loader)

<module 'etl_dwh_loader' from 'c:\\Users\\samaz\\Desktop\\TFF\\MarketReviews_VS_TRUTH\\etl_dwh_loader.py'>

In [5]:
print("Staging -> DWH) started")

conn_staging_read = None
cursor_staging_read = None
tasks = []

try:
    # 1. Connexion initiale au Staging (juste pour lire la liste des tâches)
    conn_staging_read = etl_loader.get_staging_connection()
    cursor_staging_read = conn_staging_read.cursor()
    
    # 2. Extraire les tâches (E)
    tasks = etl_loader.get_pending_cleansed_data(cursor_staging_read)
    
    cursor_staging_read.close()
    conn_staging_read.close()
    
    if not tasks:
        print("\n--- ✅ Fin : Rien à charger dans le DWH. ---")
    else:
        print(f"\n--- {len(tasks)} produits à charger trouvés. Lancement de la boucle... ---")

    # 3. Boucle de Transformation (T) et Chargement (L)
    # On traite chaque produit comme sa propre transaction
    for task in tasks:
        cleansed_id = task['CleansedProductID']
        sku = task['SKU']
        
        # On a besoin de deux connexions par boucle pour gérer les transactions séparément
        conn_dwh = None
        cursor_dwh = None
        conn_staging_write = None
        cursor_staging_write = None
        
        try:
            print(f"\n--- Traitement du SKU {sku} (CleansedID: {cleansed_id}) ---")
            
            # --- Transaction DWH ---
            conn_dwh = etl_loader.get_dwh_connection()
            cursor_dwh = conn_dwh.cursor()
            
            # 3a. Gérer la Dimension (SCD Type 1)
            product_key = etl_loader.load_dim_product_scd1(cursor_dwh, task)
            
            # 3b. Gérer les Faits
            date_key, time_key = etl_loader.get_date_time_keys(task['ScrapeTimestamp'])
            etl_loader.load_fact_snapshot(cursor_dwh, task, product_key, date_key, time_key)
            
            conn_dwh.commit() # Si tout s'est bien passé dans le DWH, on valide
            print(f"   -> ✅ SKU {sku} chargé dans le DWH.")

            # --- Transaction Staging ---
            # Si le DWH est OK, on met à jour le statut du Staging
            conn_staging_write = etl_loader.get_staging_connection()
            cursor_staging_write = conn_staging_write.cursor()
            
            etl_loader.update_staging_status(cursor_staging_write, cleansed_id, 'processed')
            conn_staging_write.commit() # Valide le statut 'processed'

        except Exception as e:
            # Gérer une erreur pour ce produit spécifique
            print(f"   -> ❌ ERREUR sur SKU {sku}: {e}.")
            if conn_dwh: conn_dwh.rollback()     # Annuler les changements dans le DWH
            if conn_staging_write: conn_staging_write.rollback() # Annuler
            
            # Marquer comme 'failed' (dans une transaction séparée)
            conn_fail = etl_loader.get_staging_connection()
            cursor_fail = conn_fail.cursor()
            etl_loader.update_staging_status(cursor_fail, cleansed_id, 'failed')
            conn_fail.commit()
            cursor_fail.close()
            conn_fail.close()

        finally:
            if cursor_staging_write: cursor_staging_write.close()
            if conn_staging_write: conn_staging_write.close()
            if cursor_dwh: cursor_dwh.close()
            if conn_dwh: conn_dwh.close()

except Exception as e_main:
    print(f"\n--- ❌ ERREUR MAJEURE du Pipeline : {e_main} ---")
finally:
    # Au cas où la connexion de lecture initiale planterait
    print("\nScript terminé.")

Staging -> DWH) started
   -> (E) Extraction des données de Staging_Product_Cleansed (Status='pending')...
   -> 0 produits trouvés à migrer.

--- ✅ Fin : Rien à charger dans le DWH. ---

Script terminé.
